# Clustering com K-Means, DBSCA e Mean Shift

In [ ]:
import random
import joblib
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN, MeanShift, estimate_bandwidth
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, silhouette_score
from sklearn.decomposition import PCA
from kneed import KneeLocator
from scipy.spatial import distance
from itertools import cycle

import warnings

warnings.filterwarnings("ignore")
%matplotlib inline

### Carregando Dados

In [ ]:
arquivo_excel = 'dados/Operacoes.xlsx'

df_realizado = pd.read_excel(arquivo_excel, sheet_name='Realizado', header=1, index_col=0)

In [ ]:
df_realizado.shape

In [ ]:
df_realizado.columns

In [ ]:
df_realizado

### Análise Exploratória

In [ ]:
# Tipos dos Dados
df_realizado.dtypes

#### - Variáveis Numéricas

In [ ]:
df_realizado.describe()

In [ ]:
#Total Clientes
total_cliente = len(df_realizado['Cliente'].unique())
total_cliente

In [ ]:
#Criando medidas de RFM para apoiar na clusterização

datareferencia = dt.datetime(2024, 10, 25)

rfm = df_realizado.groupby('Cliente').agg({
    'Data' : lambda x: (datareferencia - x.max()).days,
     'Cliente' : 'count',
    'Saldo Devedor Inicial' : 'sum'
}). rename(columns={'Data':'Recencia', 'Cliente':'Frequencia', 'Saldo Devedor Inicial':'Monetario'}).reset_index()

rfm

In [ ]:
rfm['R_score'] = pd.cut(rfm['Recencia'], 4, labels=[4, 3, 2, 1], include_lowest=True)
rfm['F_score'] = pd.cut(rfm['Frequencia'], 4, labels=[1, 2, 3, 4], include_lowest=True)
rfm['M_score'] = pd.cut(rfm['Monetario'], 4, labels=[1, 2, 3, 4], include_lowest=True)

rfm['RFM_Score'] = rfm['R_score'].astype(int) + rfm['F_score'].astype(int) + rfm['M_score'].astype(int)

rfm

In [ ]:
df_realizado = pd.merge(df_realizado, rfm[['Cliente', 'RFM_Score']], on='Cliente', how='left')

df_realizado

In [ ]:
#Criando a coluna Renda Estimada do cliente

renda = df_realizado.groupby('Cliente').agg({
     'PMT' : lambda x: (sum(x * 4))
}).rename(columns={'PMT':'Renda Mensal Estimada'}).reset_index()

renda

In [ ]:
df_realizado = pd.merge(df_realizado, renda[['Cliente', 'Renda Mensal Estimada']], on='Cliente', how='left')

df_realizado

In [ ]:
datareferencia = dt.datetime(2024, 11, 25)

df_realizado['Data'] = df_realizado['Data'].apply(lambda x: (datareferencia - x).days)

df_realizado

In [ ]:
df_realizado.hist(figsize = (15,15), bins = 10)
plt.show()

In [ ]:
df_corr = df_realizado.select_dtypes(exclude=['object'])

correlation_matrix = df_corr.corr()

fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', cbar=True, square=True)
ax.set_title('Matriz de Correlação')

#salvando figura
#fig.savefig('corr.png', format='png')
plt.show() 

#### - Variáveis Categoricas

In [ ]:
df_realizado.describe(include = ['object'])

In [ ]:
sns.countplot(data = df_realizado, x = 'Produto')

CT, S, CO, CP, P = df_realizado['Produto'].value_counts()

print('Numero de portabilidade de Cartão: ', CT)
print('Numero de portabilidade de Seguro: ', S)
print('Numero de portabilidade de Pix: ', P)
print('Numero de portabilidade de Consignado: ', CO)
print('Numero de portabilidade de Crédito Pessoal: ', CP)

In [ ]:
#Enconding Variave Produto

mapping_dic = { 'Cartão' : 1,
                'Seguro' : 2,
                'Pix' : 3,
                'Consignado' : 4,
                'Crédito Pessoal' : 5
              }


df_realizado['Produto'] = df_realizado.Produto.map(mapping_dic)

df_realizado

In [ ]:
df_realizado[df_realizado.isnull().values]

In [ ]:
df_realizado[df_realizado.duplicated(keep = False)]

### Pré Processamento Normalização

In [ ]:
df_realizado_norm_km = df_realizado.drop(columns=['Cliente'])
df_realizado_norm_db = df_realizado_norm_km.copy()
df_realizado_norm_ms = df_realizado_norm_km.copy()
df_realizado_norm_km

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

df_realizado_norm_km = scaler.fit_transform(df_realizado_norm_km)
df_realizado_norm_db = scaler.fit_transform(df_realizado_norm_db)
df_realizado_norm_ms = scaler.fit_transform(df_realizado_norm_ms)

print("Normalização Min-Max:\n", df_realizado_norm_km)

#### Identificado se o DataFrame é clusterizavel

In [ ]:
# Aplicando Hopkins para identificar se o dataframe é clusterizavel
def hopkins_statistic(data, n=None):

    if n is None:
        n = len(data)

    data = np.array(data)
    
    random_indices = random.sample(range(len(data)), n)
    random_points = data[random_indices]
    
    d1 = []
    d2 = []
    
    for point in random_points:
        distances = distance.cdist([point], data, 'euclidean')[0]
        sorted_distances = np.sort(distances) 
        d1.append(sorted_distances[1]) 
        d2.append(sorted_distances[2])  

    D1_sum = np.sum(d1)
    D2_sum = np.sum(d2)
    
    H = D1_sum / (D1_sum + D2_sum)
    
    return H

In [ ]:
# Ideal valores acima de 0.75, 0.5 indica leve tendencia de agrupamento nos dados, mas nada muito claro ou estruturado, problema com Silhouette Score
hopkins = hopkins_statistic(df_realizado_norm_km)

print(f'Índice de Hopkins: {hopkins}')

#### Identificando o número ideal de cluster

In [ ]:
# Inercia - Soma da distancia em relação ao centro
Ks = range(2, 11) 

valor_metrica = []

for k in Ks:
    modelo = KMeans(n_clusters=k, random_state = 101)
    modelo.fit(df_realizado_norm_km)
    valor_metrica.append(modelo.inertia_)

plt.plot(Ks, valor_metrica, 'o-')
plt.xlabel('Valor de K')
plt.ylabel('Inertia')
plt.show()

In [ ]:
# Método de Elbow

inertia = []
range_n_clusters = range(1, 11) 

for k in range_n_clusters:
    kmeans = KMeans(n_clusters = k,
                    init = 'k-means++',
                    random_state=111)
    kmeans.fit(df_realizado_norm_km)
    inertia.append(kmeans.inertia_)

knee_locator = KneeLocator(range_n_clusters, inertia, curve="convex", direction="decreasing")
optimal_k = knee_locator.knee

print(f"O número ideal de clusters é: {optimal_k}")

plt.figure(figsize=(8, 5))
plt.plot(range_n_clusters, inertia, marker="o", linestyle="--", color="b", label="Inertia")
plt.axvline(x=optimal_k, color="r", linestyle="--", label=f"Optimal K = {optimal_k}")
plt.title("Método Elbow")
plt.xlabel("Número de Clusters (k)")
plt.ylabel("Inertia")
plt.legend()
plt.show()

In [ ]:
# K-Means para gerar pseudo-rótulos com a quantidade de cluster indicado no metodo de elbow
df_realizado2 = df_realizado.copy()
df_realizado_norm2 = df_realizado2.drop(columns=["Cliente"])

kmeans = KMeans(n_clusters=3, random_state=42)
pseudo_labels = kmeans.fit_predict(df_realizado_norm2) 
df_realizado_norm2['labels'] = pseudo_labels

print(df_realizado_norm2.head())

In [ ]:
# Homogeneidade - pontos de dados do cluster membros de uma unica classe
# Completude - pontos de dados de uma classe membros do mesmo cluster
# V Measure - media homogeneidade / completude

X = df_realizado_norm2.drop(columns=["labels"]) 
y_pred = df_realizado_norm2["labels"]  

scaler = MinMaxScaler(feature_range=(0, 1))
df_realizado_norm2 = scaler.fit_transform(df_realizado_norm2)
#print("Normalização Min-Max:\n", df_realizado_norm)

# Testando diferentes números de clusters
range_n_clusters = range(2, 11)
homogeneity = []
completeness = []
v_measure = []

for k in range_n_clusters:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(df_realizado_norm2)
    
    # Calcular as métricas
    homogeneity.append(homogeneity_score(y_pred, labels))
    completeness.append(completeness_score(y_pred, labels))
    v_measure.append(v_measure_score(y_pred, labels))

# Plotar as métricas
plt.figure(figsize=(8, 5))
plt.plot(range_n_clusters, homogeneity, label="Homogeneidade", marker="o", color="b")
plt.plot(range_n_clusters, completeness, label="Completude", marker="o", color="g")
plt.plot(range_n_clusters, v_measure, label="Medida V", marker="o", color="r")
plt.title("Homogeneidade, Completude e Medida V por Número de Clusters")
plt.xlabel("Número de Clusters")
plt.ylabel("Métricas")
plt.legend()
plt.grid(True)
plt.show()


#### Modelo de Clustering

##### K- Means

In [ ]:
kmeans = KMeans(n_clusters = 3,
                init = 'k-means++',
                random_state = 111)
kmeans.fit(df_realizado_norm_km)

In [ ]:
n_clusters = len(set(kmeans.labels_))
n_clusters , kmeans.labels_

In [ ]:
pca = PCA(n_components = 2).fit(df_realizado_norm_km)
pca2d = pca.transform(df_realizado_norm_km)

for i in range(0, pca2d.shape[0]):
    if kmeans.labels_[i] == 0:
        c1 = plt.scatter(pca2d[i,0],pca2d[i,1], c = 'r', marker = '+')

    elif kmeans.labels_[i] == 1:
        c2 = plt.scatter(pca2d[i,0],pca2d[i,1], c = 'g', marker = 'o')

    elif kmeans.labels_[i] == 2:
        c3 = plt.scatter(pca2d[i,0],pca2d[i,1], c = 'b', marker = '*')
        
plt.legend([c1,c2,c3],['Cluster 0', 'Cluster 1', 'Cluster 2'])
plt.title('Cluster K-means - Número de Clusters: %d' % n_clusters)

plt.show()

In [ ]:
score = silhouette_score(df_realizado_norm_km, kmeans.labels_ )
if score > 0.5:
    obs = 'Boa clusterização pois o Silhoutte Score é > 0.5'
else:
    obs = 'Silhoutte Score ficar < 0.5 recomenda-se reavaliar o dataframe ou as métricas, para alcançar > 0.5'
    
score

In [ ]:
df_modelos = pd.DataFrame()

dict_kmeans = {'Algoritmo' : 'KMeans',
               'Silhouette Score' : score,
               'OBS' : obs}

new_row = pd.DataFrame([dict_kmeans])

df_modelos = pd.concat([df_modelos, new_row], ignore_index=True)
df_modelos

##### DBSCAN

In [ ]:
dbscan_v1 = DBSCAN(eps=0.3, min_samples=3)

dbscan_v1.fit(df_realizado_norm_db)

In [ ]:
labels = dbscan_v1.labels_[dbscan_v1.labels_ != -1]
n_clusters = len(set(dbscan_v1.labels_)) - (1 if -1 in dbscan_v1.labels_ else 0)

n_clusters, labels

In [ ]:
pca = PCA(n_components=2).fit(df_realizado_norm_db)
pca2d = pca.transform(df_realizado_norm_db)

unique_labels = sorted(set(dbscan_v1.labels_))
n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)

colors = list(plt.cm.tab20.colors) + list(plt.cm.Paired.colors) 
markers = ['o', 's', 'D', '^', 'v', 'P', '*', 'X', '<', '>', 'h', '+'] * 3 

colors = colors * (len(unique_labels) // len(colors) + 1)
markers = markers * (len(unique_labels) // len(markers) + 1)

label_to_index = {label: idx for idx, label in enumerate(unique_labels)}

for cluster_id in unique_labels:
    if cluster_id == -1:  # Tratar ruído separadamente
        plt.scatter(pca2d[dbscan_v1.labels_ == -1, 0],
                    pca2d[dbscan_v1.labels_ == -1, 1],
                    c='k', marker='x', label='Noise')
    else:
        idx = dbscan_v1.labels_ == cluster_id
        color_idx = label_to_index[cluster_id]
        plt.scatter(pca2d[idx, 0],
                    pca2d[idx, 1],
                    c=[colors[color_idx]],
                    marker=markers[color_idx],
                    label=f'Cluster {cluster_id}')
        
plt.legend()
plt.title('Cluster DBSCAN - Número estimado de Clusters: %d' % n_clusters)

plt.show()

In [ ]:
if n_clusters > 1:  
    score = silhouette_score(df_realizado_norm_km, dbscan_v1.labels_)
    print(f"Silhouette Score (excluindo outliers): {score}")
else:
    score = 0
    obs = "Silhouette Score não pode ser calculado com menos de 2 clusters válidos."
    print("Silhouette Score não pode ser calculado com menos de 2 clusters válidos.")

In [ ]:
dict_DBSCAN = {'Algoritmo' : 'DBSCAN',
               'Silhouette Score' : score,
               'OBS' : obs}

new_row = pd.DataFrame([dict_DBSCAN])

df_modelos = pd.concat([df_modelos, new_row], ignore_index=True)
df_modelos

##### Mean Shift

In [ ]:
bandwidth = estimate_bandwidth(df_realizado_norm_ms, quantile = .1,  n_samples = 500, )

meanshift_v1 = MeanShift(bandwidth = bandwidth, bin_seeding = True)

meanshift_v1.fit(df_realizado_norm_ms)

In [ ]:
labels = meanshift_v1.labels_[meanshift_v1.labels_ != -1]
n_clusters = len(np.unique(labels))

n_clusters, labels

In [ ]:
pca = PCA(n_components = 2).fit(df_realizado_norm_ms)
pca2d = pca.transform(df_realizado_norm_ms)

for i in range(0, pca2d.shape[0]):
    if meanshift_v1.labels_[i] == 0:
        c1 = plt.scatter(pca2d[i,0],pca2d[i,1], c = 'r', marker = '+')
        
    elif meanshift_v1.labels_[i] == 1:
         c2 = plt.scatter(pca2d[i,0],pca2d[i,1], c = 'g', marker = 'o')
        
    elif meanshift_v1.labels_[i] == 2:
        c3 = plt.scatter(pca2d[i,0],pca2d[i,1], c = 'b', marker = '*')
        

plt.title('Cluster Mean Shift - Número estimado de Clusters: %d' % n_clusters)

plt.show()

In [ ]:
if n_clusters > 1: 
    score = silhouette_score(df_realizado_norm_ms, labels)
    print(f"Silhouette Score: {score}")
else:
    score = 0
    obs = "Silhouette Score não pode ser calculado com menos de 2 clusters válidos."
    print("Silhouette Score não pode ser calculado com menos de 2 clusters.")

In [ ]:
dict_MeanShift = {'Algoritmo' : 'Mean Shift',
                  'Silhouette Score' : score,
                  'OBS' : obs}

new_row = pd.DataFrame([dict_MeanShift])

df_modelos = pd.concat([df_modelos, new_row], ignore_index=True)

#### Avaliação dos Modelos

In [ ]:
df_modelos

In [ ]:
# Desvio Padrão baixo em relação a distancia indica pouca variabilidade espacial, dados não heterogêneo, distribuidos de maneira uniforme no espaço multidimensional, não apresenta agrupamentos naturais claros
distances = distance.pdist(df_realizado_norm_km, metric='euclidean')
print(f"Distância média: {np.mean(distances):.4f}, Desvio padrão: {np.std(distances):.4f}")

In [ ]:
df_realizado.groupby('Produto').agg({
     'Renda Mensal Estimada' : 'mean'
}).reset_index()

In [ ]:
df_realizado.groupby('Produto').agg({
     'PMT' : 'mean'
}).reset_index()